# MultiVectorRetriever

- Author: [YooKyung Jeon](https://github.com/sirena1)
- Peer Review: [choincnp](https://github.com/choincnp), [Hye-yoonJeong](https://github.com/Hye-yoonJeong)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/99-TEMPLATE/00-BASE-TEMPLATE-EXAMPLE.ipynb)

## Overview

In LangChain, there's a special feature called `MultiVectorRetriever` that enables efficient querying of documents in various contexts. This feature allows documents to be stored and managed with multiple vectors, significantly enhancing the accuracy and efficiency of information retrieval.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Methods for Generating Multiple Vectors Per Document](#methods-for-generating-multiple-vectors-per-document)
- [Document Used for Practice](#document-used-for-practice)
- [Chunk + Original Document Retrieval](#chunk--original-document-retrieval)
- [Storing summaries in vector storage](#storing-summaries-in-vector-storage)
- [Utilizing Hypothetical Queries to explore document content](#utilizing-hypothetical-queries-to-explore-document-content)

### References

- [Software Policy Research Institute (SPRi) - December 2023 Issue](https://spri.kr/posts/view/23669)
----

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [3]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [13]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langchain_community",
        "langchain",
        "langchain_chroma",
        "langchain_openai",
        "langchain_core",
        "langchain_text_splitters",
    ],
    verbose=False,
    upgrade=False,
)

In [2]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [ ]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "09-FewShot-Prompt-Templates",
    }
)

## Methods for Generating Multiple Vectors Per Document

1. **Creating Small Chunks**: Divide the document into smaller chunks and generate separate embeddings for each chunk. This method enables a more granular focus on specific parts of the document. It can be implemented using the `ParentDocumentRetriever`, making it easier to explore detailed information.

2. **Summary Embeddings**: Generate a summary for each document and create embeddings based on this summary. Summary embeddings are particularly useful for quickly grasping the core content of a document. By focusing only on the summary instead of analyzing the entire document, efficiency can be significantly improved.

3. **Utilizing Hypothetical Questions**: Create relevant hypothetical questions for each document and generate embeddings based on these questions. This approach is helpful when deeper exploration of specific topics or content is needed. Hypothetical questions enable a broader perspective on the document's content, facilitating a more comprehensive understanding.

4. **Manual Addition**: Users can manually add specific questions or queries that should be considered during document retrieval. This method provides users with more control over the search process, allowing for customized searches tailored to their specific needs.


## Document Used for Practice

Software Policy Research Institute (SPRi) - December 2023 Issue

- Authors: Jaeheung Yoo (Senior Researcher, AI Policy Research Division), Jisoo Lee (Research Fellow, AI Policy Research Division)
- Link: https://spri.kr/posts/view/23669
- File Name: `SPRI_AI_Brief_2023년12월호_F.pdf`

**Note**: Download the file into the `data` folder for reference.

The preprocessing process involves loading data from a text file and splitting the loaded documents into specified sizes.

The split documents can later be used for tasks such as vectorization and retrieval.

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
docs = loader.load()

The original documents loaded from the data are stored in the docs variable.

In [4]:
print(docs[5].page_content[:500])

1. 정책/법제  
2. 기업/산업 
3. 기술/연구 
 4. 인력/교육
영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언
n 영국 블레츨리 파크에서 개최된 AI 안전성 정상회의에 참가한 28개국들이 AI 안전 보장을 
위한 협력 방안을 담은 블레츨리 선언을 발표
n 첨단 AI를 개발하는 국가와 기업들은 AI 시스템에 대한 안전 테스트 계획에 합의했으며, 
영국의 AI 안전 연구소가 전 세계 국가와 협력해 테스트를 주도할 예정 
KEY Contents
£ AI 안전성 정상회의 참가국들, 블레츨리 선언 통해 AI 안전 보장을 위한 협력에 합의
n 2023년 11월 1~2일 영국 블레츨리 파크에서 열린 AI 안전성 정상회의(AI Safety Summit)에 
참가한 28개국 대표들이 AI 위험 관리를 위한 ‘블레츨리 선언’을 발표 
∙선언은 AI 안전 보장을 위해 국가, 국제기구, 기업, 시민사회, 학계를 포함한 모든 이해관계자의 협력이 
중요하다고 강조했으며,


## Chunk + Original Document Retrieval

When searching through large volumes of information, embedding data into smaller chunks can be highly beneficial.

With `MultiVectorRetriever`, documents can be stored and managed as multiple vectors.

- The original documents are stored in the `docstore`.
- The embedded documents are stored in the `vectorstore`.

This allows for splitting documents into smaller units, enabling more accurate searches. Additionally, the contents of the original document can be accessed when needed.


In [5]:
import uuid
from langchain.storage import InMemoryByteStore
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

# Vector store for indexing child chunks
vectorstore = Chroma(
    collection_name="small_bigger_chunks",
    embedding_function=OpenAIEmbeddings(model="text-embedding-ada-002"),
)

# Storage layer for parent documents
store = InMemoryByteStore()

id_key = "doc_id"

# Retriever (initially empty)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

# Generate document IDs
doc_ids = [str(uuid.uuid4()) for _ in docs]

# Verify two of the generated IDs
print(doc_ids[:2])

['09bd23c4-43fc-460c-930d-a1c67a887444', '7d575c47-aff1-4033-9c6b-cb4538d820e9']


Defining `parent_text_splitter` for Larger Chunks and `child_text_splitter` for Smaller Chunks

Here, we define `parent_text_splitter` for splitting into larger chunks and `child_text_splitter` for splitting into smaller chunks.


In [6]:
# Create a RecursiveCharacterTextSplitter object for larger chunks
parent_text_splitter = RecursiveCharacterTextSplitter(chunk_size=600)

# Splitter to be used for generating smaller chunks
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)

Create Parent documents as larger chunks.

In [7]:
parent_docs = []

for i, doc in enumerate(docs):
    # Retrieve the ID of the current document
    _id = doc_ids[i]
    # Split the current document into smaller parent documents
    parent_doc = parent_text_splitter.split_documents([doc])

    for _doc in parent_doc:
        # Store the document ID in the metadata
        _doc.metadata[id_key] = _id
    parent_docs.extend(parent_doc)

Verify the `doc_id` assigned to `parent_docs`


In [8]:
# Check the metadata of the generated Parent documents.
parent_docs[0].metadata

{'source': '.\\data\\SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': '.\\data\\SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 0,
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13462',
 'producer': 'Hancom PDF 1.3.0.542',
 'creationDate': "D:20231208132838+09'00'",
 'modDate': "D:20231208132838+09'00'",
 'trapped': '',
 'doc_id': '09bd23c4-43fc-460c-930d-a1c67a887444'}

Create Child documents as relatively smaller chunks.

In [9]:
child_docs = []
for i, doc in enumerate(docs):
    # Retrieve the ID of the current document
    _id = doc_ids[i]
    # Split the current document into child documents
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:
        # Store the document ID in the metadata
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)

Verify the `doc_id` assigned to `child_docs`.

In [10]:
# Check the metadata of the generated Child documents.
child_docs[0].metadata

{'source': '.\\data\\SPRI_AI_Brief_2023년12월호_F.pdf',
 'file_path': '.\\data\\SPRI_AI_Brief_2023년12월호_F.pdf',
 'page': 0,
 'total_pages': 23,
 'format': 'PDF 1.4',
 'title': '',
 'author': 'dj',
 'subject': '',
 'keywords': '',
 'creator': 'Hwp 2018 10.0.0.13462',
 'producer': 'Hancom PDF 1.3.0.542',
 'creationDate': "D:20231208132838+09'00'",
 'modDate': "D:20231208132838+09'00'",
 'trapped': '',
 'doc_id': '09bd23c4-43fc-460c-930d-a1c67a887444'}

Check the number of chunks for each split document.

In [11]:
print(f"Number of split parent_docs: {len(parent_docs)}")
print(f"Number of split child_docs: {len(child_docs)}")

Number of split parent_docs: 74
Number of split child_docs: 442


Add the newly created smaller child document set to the vector store

Next, map the parent documents to the generated UUIDs and add them to the `docstore`.

- Use the `mset()` method to store document IDs and their content as key-value pairs in the document store.

In [12]:
# Add both parent and child documents to the vector store
retriever.vectorstore.add_documents(parent_docs)
retriever.vectorstore.add_documents(child_docs)

# Store the original documents in the docstore
retriever.docstore.mset(list(zip(doc_ids, docs)))

Perform Similarity Search and Display the Most Similar Document Chunk

Use the `retriever.vectorstore.similarity_search` method to search within child and parent document chunks.

The first document chunk with the highest similarity will be displayed.

In [13]:
# Perform similarity search on the vectorstore
relevant_chunks = retriever.vectorstore.similarity_search(
    "What is the name of the generative AI created by Samsung Electronics?"
)
print(f"Number of retrieved documents: {len(relevant_chunks)}")

Number of retrieved documents: 4


In [14]:
for chunk in relevant_chunks:
    print(chunk.page_content, end="\n\n")
    print(">" * 100, end="\n\n")

☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2를 탑재한 퀄컴 기기 및 구글 
어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상
☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동

Execute a Query Using the `retriever.invoke()` Method

The `retriever.invoke()` method performs a search across the full content of the original documents.

In [15]:
relevant_docs = retriever.invoke(
    "What is the name of the generative AI created by Samsung Electronics?"
)
print(f"Number of retrieved documents: {len(relevant_docs)}", end="\n\n")
print("=" * 100, end="\n\n")
print(relevant_docs[0].page_content)

Number of retrieved documents: 2


SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 


The default search type performed by the retriever in the vector database is similarity search.

LangChain Vector Stores also support searching using [Max Marginal Relevance](https://api.python.langchain.com/en/latest/vectorstores/langchain_core.vectorstores.VectorStore.html#langchain_core.vectorstores.VectorStore.max_marginal_relevance_search). 

If you want to use this method instead, you can configure the `search_type` property as follows.

- Set the `search_type` property of the `retriever` object to `SearchType.mmr`.
  - This specifies that the MMR (Maximal Marginal Relevance) algorithm should be used during the search.

In [16]:
from langchain.retrievers.multi_vector import SearchType

# Set the search type to Maximal Marginal Relevance (MMR)
retriever.search_type = SearchType.mmr

# Search all related documents
print(
    retriever.invoke(
        "What is the name of the generative AI created by Samsung Electronics?"
    )[0].page_content
)

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Ll

In [17]:
from langchain.retrievers.multi_vector import SearchType

# Set search type to similarity_score_threshold
retriever.search_type = SearchType.similarity_score_threshold
retriever.search_kwargs = {"score_threshold": 0.3}

# Search all related documents
print(
    retriever.invoke(
        "What is the name of the generative AI created by Samsung Electronics?"
    )[0].page_content
)

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있도록 지원하며 
저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Ll

In [18]:
from langchain.retrievers.multi_vector import SearchType

# Set search type to similarity and k value to 1
retriever.search_type = SearchType.similarity
retriever.search_kwargs = {"k": 1}

# Search all related documents
print(
    len(
        retriever.invoke(
            "What is the name of the generative AI created by Samsung Electronics?"
        )
    )
)

1


## Storing summaries in vector storage

Summaries can often provide a more accurate extraction of the contents of a chunk, which can lead to better search results.

This section describes how to generate summaries and how to embed them.

In [19]:
# Importing libraries for loading PDF files and splitting text
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Initialize the PDF file loader
loader = PyMuPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")

# Split text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)

# Load a PDF file and run Text Split
split_docs = loader.load_and_split(text_splitter)

# Output the number of split documents
print(f"Number of split documents: {len(split_docs)}")

Number of split documents: 61


In [20]:
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


summary_chain = (
    {"doc": lambda x: x.page_content}
    # Create a prompt template for document summaries
    | ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert in summarizing documents in Korean."),
            (
                "user",
                "Summarize the following documents in 3 sentences in bullet points format.\n\n{doc}",
            ),
        ]
    )
    # Using OpenAI's ChatGPT model to generate summaries
    | ChatOpenAI(temperature=0, model="gpt-4o-mini")
    | StrOutputParser()
)

Summarize the documents in the `docs` list in batch using the `chain.batch` method.
- Here, we set the `max_concurrency` parameter to 10 to allow up to 10 documents to be processed simultaneously.

In [21]:
# Handling batches of documents
summaries = summary_chain.batch(split_docs, {"max_concurrency": 10})

In [22]:
len(summaries)

61

Print the summary to see the results.

In [23]:
# Prints the contents of the original document.
print(split_docs[33].page_content, end="\n\n")
# Print a summary.
print("[summary]")
print(summaries[33])

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에

[summary]
- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이 모델은 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


Initialize the `Chroma` vector store to index the child chunks. Use `OpenAIEmbeddings` as the embedding function.

- Use `“doc_id”` as the key representing the document ID.


In [25]:
import uuid

# Create a vector store to store the summary information.
summary_vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small"),
)

# Create a repository to store the parent document.
store = InMemoryByteStore()

# Specify a key name to store the document ID.
id_key = "doc_id"

# Initialize the searcher (empty at startup).
retriever = MultiVectorRetriever(
    vectorstore=summary_vectorstore,  # vector store
    byte_store=store,  # byte store
    id_key=id_key,  # document ID
)
# Create a document ID.
doc_ids = [str(uuid.uuid4()) for _ in split_docs]

Save the summarized document and its metadata (here, the `Document ID` for the summary you created).


In [26]:
summary_docs = [
    # Create a Document object with the summary as the page content and the document ID as metadata.
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]

The number of articles in the digest matches the number of original articles.


In [27]:
# Number of documents in the summary
len(summary_docs)

61

- Add `summary_docs` to the vector store with `retriever.vectorstore.add_documents(summary_docs)`.
- Map `doc_ids` and `docs` with `retriever.docstore.mset(list(zip(doc_ids, docs))))` to store them in the document store.


In [28]:
retriever.vectorstore.add_documents(
    summary_docs
)  # Add the summarized document to the vector repository.

# Map the document ID to the document and store it in the document store.
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

Perform a similarity search using the `similarity_search` method of the `vectorstore` object.


In [29]:
# Perform a similarity search.
result_docs = summary_vectorstore.similarity_search(
    "What is the name of the generative AI created by Samsung Electronics?"
)

In [30]:
# Output 1 result document.
print(result_docs[0].page_content)

- 삼성전자가 온디바이스에서 작동 가능한 생성 AI 모델 '삼성 가우스'를 공개하였으며, 이 모델은 언어, 코드, 이미지의 3개 모델로 구성되어 있다.
- '삼성 가우스'는 정규분포 이론을 정립한 수학자 가우스의 이름을 따왔으며, 다양한 상황에 최적화된 모델 선택이 가능하다.
- 삼성전자는 이 AI 모델이 사용자 정보를 외부로 유출하지 않도록 설계되었으며, 향후 다양한 제품에 단계적으로 탑재할 계획이다.


Use the `invoke()` of the `retriever` object to retrieve documents related to your question.


In [31]:
# Search for and fetch related articles.
retrieved_docs = retriever.invoke(
    "What is the name of the generative AI created by Samsung Electronics?"
)
print(retrieved_docs[0].page_content)

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에


## Utilizing Hypothetical Queries to explore document content

LLM can also be used to generate a list of questions that can be hypothesized about a particular document.

These generated questions can be embedded to further explore and understand the content of the document.

Generating hypothetical questions can help you identify key topics and concepts in your documentation, and can encourage readers to ask more questions about the content of your documentation.


Below is an example of creating a hypothesis question utilizing `Function Calling`.

In [32]:
functions = [
    {
        "name": "hypothetical_questions",  # Specify a name for the function.
        "description": "Generate hypothetical questions",  # Write a description of the function.
        "parameters": {  # Define the parameters of the function.
            "type": "object",  # Specifies the type of the parameter as an object.
            "properties": {  # Defines the properties of an object.
                "questions": {  # Define the 'questions' attribute.
                    "type": "array",  # Type 'questions' as an array.
                    "items": {
                        "type": "string"
                    },  # Specifies the array's element type as String.
                },
            },
            "required": ["questions"],  # Specify 'questions' as a required parameter.
        },
    }
]

Use `ChatPromptTemplate` to define a prompt template that generates three hypothetical questions based on the given document.

- Set `functions` and `function_call` to call the virtual question generation functions.
- Use `JsonKeyOutputFunctionsParser` to parse the generated virtual questions and extract the values corresponding to the `questions` key.

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain_openai import ChatOpenAI

hypothetical_query_chain = (
    {"doc": lambda x: x.page_content}
    # We ask you to create exactly 3 hypothetical questions that you can answer using the documentation below. This number can be adjusted.
    | ChatPromptTemplate.from_template(
        "Generate a list of exactly 3 hypothetical questions that the below document could be used to answer. "
        "Potential users are those interested in the AI industry. Create questions that they would be interested in. "
        "Output should be written in Korean:\n\n{doc}"
    )
    | ChatOpenAI(max_retries=0, model="gpt-4o-mini").bind(
        functions=functions, function_call={"name": "hypothetical_questions"}
    )
    # Extract the value corresponding to the “questions” key from the output.
    | JsonKeyOutputFunctionsParser(key_name="questions")
)

Output the answers to the documents.

- The output contains the three Hypothetical Queries you created.


In [34]:
# Run the chain for the given document.
hypothetical_query_chain.invoke(split_docs[33])

['삼성 가우스가 다른 생성 AI 모델과 비교했을 때 어떤 독창적인 기능을 제공할 수 있을까?',
 '온디바이스에서 작동하는 삼성 가우스가 개인 정보 보호에 미치는 영향은 무엇일까?',
 '삼성전자가 삼성 가우스를 다양한 제품에 탑재했을 때, 사용자 경험은 어떻게 변화할까?']

Use the `chain.batch` method to process multiple requests for `split_docs` data at the same time.

In [35]:
# Create a batch of hypothetical questions for a list of articles
hypothetical_questions = hypothetical_query_chain.batch(
    split_docs, {"max_concurrency": 10}
)

In [36]:
hypothetical_questions[33]

['삼성 가우스가 다른 생성 AI 모델과 비교하여 어떤 장점이 있을까요?',
 '온디바이스에서 작동하는 생성 AI 모델이 사용자의 개인정보 보호에 어떤 기여를 할 수 있을까요?',
 '삼성전자가 삼성 가우스를 다양한 제품에 탑재할 경우, 어떤 산업에서 가장 큰 영향을 미칠까요?']

Below is the process for storing the Hypothetical Queries you created in Vector Storage, the same way we did before.


In [37]:
# Vector store to use for indexing child chunks
hypothetical_vectorstore = Chroma(
    collection_name="hypo-questions", embedding_function=OpenAIEmbeddings()
)
# Storage hierarchy for parent documents
store = InMemoryByteStore()

id_key = "doc_id"
# Retriever (empty on startup)
retriever = MultiVectorRetriever(
    vectorstore=hypothetical_vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in split_docs]  # Create a document ID

Add metadata (document IDs) to the `question_docs` list.


In [38]:
question_docs = []
# save hypothetical_questions
for i, question_list in enumerate(hypothetical_questions):
    question_docs.extend(
        # Create a Document object for each question in the list of questions, and include the document ID for that question in the metadata.
        [Document(page_content=s, metadata={id_key: doc_ids[i]}) for s in question_list]
    )

Add the hypothesized query to the document, and add the original document to `docstore`.


In [39]:
# Add the hypothetical_questions document to the vector repository.
retriever.vectorstore.add_documents(question_docs)

# Map the document ID to the document and store it in the document store.
retriever.docstore.mset(list(zip(doc_ids, split_docs)))

Perform a similarity search using the `similarity_search` method of the `vectorstore` object.


In [40]:
# Search the vector repository for similar documents.
result_docs = hypothetical_vectorstore.similarity_search(
    "What is the name of the generative AI created by Samsung Electronics?"
)

Below are the results of the similarity search.

Here, we've only added the hypothesized query we created, so it returns the documents with the highest similarity among the hypothesized queries we created.


In [41]:
# Output the results of the similarity search.
for doc in result_docs:
    print(doc.page_content)
    print(doc.metadata)

삼성전자의 생성 AI '삼성 가우스'가 향후 AI 기술 발전에 기여할 수 있는 방법은 무엇일까?
{'doc_id': '556f29e3-eb73-40c7-ad82-e6c7b71631a3'}
삼성에서 발표한 Generative AI 기술이 다른 기업들과의 경쟁에서 어떤 차별점을 제공할 수 있을까요?
{'doc_id': '353584c8-8ee6-440f-8080-349d7c493bea'}
삼성 가우스가 다른 생성 AI 모델과 비교하여 어떤 장점이 있을까요?
{'doc_id': '6ef764f2-141f-45c0-abba-e170b2cdb406'}
삼성 개발자 콘퍼런스 코리아 2023에서 발표된 내용을 바탕으로, 삼성의 AI 기술이 향후 산업에 미칠 영향은 무엇일까요?
{'doc_id': '353584c8-8ee6-440f-8080-349d7c493bea'}


Use the `invoke` method of the `retriever` object to retrieve documents related to the query.


In [42]:
# Search for and fetch related articles.
retrieved_docs = retriever.invoke(result_docs[1].page_content)

# Output the documents found.
for doc in retrieved_docs:
    print(doc.page_content)

삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에
▹ 코히어, 데이터 투명성 확보를 위한 데이터 출처 탐색기 공개  ······································· 8
   ▹ 알리바바 클라우드, 최신 LLM ‘통이치엔원 2.0’ 공개 ······················································ 9
   ▹ 삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개 ··························································· 10
   